# Neurocombat TOP, SABRE and StrokeMRI harmonized datasets

This notebook creates datasets from the above hamornized as if each is indpeendant and from a different "batch"

Note this must be run in the `neuron` environment

## import libraries

In [ ]:
import os

import pandas as pd
import numpy as np

from neuroCombat import neuroCombat

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
filepath = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath,'StrokeMRI_pvc2c.csv') 
filename_top = os.path.join(filepath,'TOP_pvc2c.csv') 
filename_sabre = os.path.join(filepath,'SABRE_pvc2_cleaned.csv') 

In [ ]:
# Now we must clean SABRE down to be like the other datasets, including dropping NAns
MRI = pd.read_csv(filename_mri)
TOP = pd.read_csv(filename_top)
SABRE =  pd.read_csv(filename_sabre)
coly = TOP.columns
SABRE = SABRE[coly]
SABRE= SABRE.dropna()
SABRE.head(3)

In [ ]:
def prep3way_for_neurocombat(dataframe1, dataframe2, dataframe3):
    """
    """
    one_selection = dataframe1.drop(['Unnamed: 0'],axis=1)
    two_selection = dataframe2.drop(['Unnamed: 0'],axis=1)
    thr_selection = dataframe3.drop(['Unnamed: 0'],axis=1)
    
    one_selection = one_selection.set_index('participant_id')
    two_selection = two_selection.set_index('participant_id')
    thr_selection = thr_selection.set_index('participant_id')
    
    one_selection = one_selection.T
    two_selection = two_selection.T
    thr_selection = thr_selection.T
    
    all_togetherF = pd.concat([one_selection, two_selection, thr_selection], axis=1, join="inner")
    print("Nan count",all_togetherF.isna().sum().sum())
    features_only = all_togetherF[2:]
    dictionary_features_len = len(features_only.T.columns)
    number = 0
    made_keys = [] 
    made_vals = [] 
    for n in features_only.T.columns:

        made_keys.append(number)
        made_vals.append(n)
        number +=1
    feature_dictF = dict(map(lambda i,j : (i,j) , made_keys,made_vals))
    ftF = features_only.reset_index()
    ftF = ftF.rename(columns={"index": "A"})
    ftF = ftF.drop(['A'], axis=1)
    ftF = ftF.dropna()
    btF = all_togetherF.reset_index()
    btF = btF.rename(columns={"index": "A"})
    btF = btF.drop(['A'], axis=1)
    btF = btF.dropna()
    len1 = len(one_selection.columns)
    len2 = len(two_selection.columns)
    len3 = len(thr_selection.columns)
    return all_togetherF, ftF, btF, feature_dictF, len1, len2, len3
    

In [ ]:
all_togetherF, ftF, btF, feature_dictF, len1, len2, len3 = prep3way_for_neurocombat(TOP, MRI, SABRE)

In [ ]:
ftF.to_csv('ftF_top_mri_sabre.csv')

In [ ]:
data = np.genfromtxt('ftF_top_mri_sabre.csv', delimiter=",", skip_header=1)
data = data[:, 1:]
data

In [ ]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * len1
middle_columns_as_two = [2] * len2
last_columns_as_three = [3] * len3
covars = {'batch':first_columns_as_one + middle_columns_as_two + last_columns_as_three,
          'sex':all_togetherF.loc['sex',:].values.tolist(),
           'age':all_togetherF.loc['age',:].values.tolist(),} 
covars = pd.DataFrame(covars) 

In [ ]:
# specifify sex as categorical
categorical_cols = ['sex']
# To specify the name of the variable that encodes for the scanner/batch covariate:
batch_col_mine = 'batch'
our_continuous_col=['age']
#Harmonization step:
data_combat = neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col_mine,
    continuous_cols=our_continuous_col,
    categorical_cols=categorical_cols)["data"]


In [ ]:
neurocombat = pd.DataFrame(data_combat)
#neurocombat

In [ ]:
def make_topper(btF, row0,row1):
    """
    This function makes top rows for something harmonized
    out of the btF part produced with prep_for_neurocombat(dataframename1, dataframename2)
    """
    topperF = btF.head(2)
    topperF = topperF.rename_axis(None, axis="columns")
    topperF = topperF.reset_index(drop=False)
    topperF = topperF.rename(columns={"index": "char"})
    topperF['char'][0] = row0 # 'age'
    topperF['char'][1] = row1 # 'sex'
    return topperF

In [ ]:
topperF = make_topper(btF,'age', 'sex')

In [ ]:
bottom = neurocombat.reset_index(drop=False)
bottom = bottom.rename(columns={"index": "char"})
bottom.columns = topperF.columns

In [ ]:
back_together = pd.concat([topperF, bottom])
back_together = back_together.T
#back_together

In [ ]:
new_header = back_together.iloc[0] #grab the first row for the header
back_together.columns = new_header #set the header row as the df header
back_together = back_together[1:]
#back_together

In [ ]:
neuro_harm_top =back_together.head(len1)
neuro_harm_mri =back_together.head(len1 + len2).tail(len2)
neuro_harm_sabre =back_together.tail(len3)

In [ ]:
neuro_harm_top = neuro_harm_top.rename(feature_dictF, axis='columns')
neuro_harm_mri = neuro_harm_mri.rename(feature_dictF, axis='columns')
neuro_harm_sabre = neuro_harm_sabre.rename(feature_dictF, axis='columns')

In [ ]:
neuro_harm_top = neuro_harm_top.reset_index()
neuro_harm_mri = neuro_harm_mri.reset_index()
neuro_harm_sabre = neuro_harm_sabre.reset_index()

neuro_harm_top = neuro_harm_top.rename(columns={'index':'participant_id'})
neuro_harm_mri = neuro_harm_mri.rename(columns={'index':'participant_id'})
neuro_harm_sabre = neuro_harm_sabre.rename(columns={'index':'participant_id'})

neuro_harm_mri

In [ ]:
# save off newly harmonized sets

In [ ]:
neuro_harm_mri.to_csv('neuro_harm3way_mri.csv')
neuro_harm_top.to_csv('neuro_harm3way_top.csv')
neuro_harm_sabre.to_csv('neuro_harm3way_sabre.csv')

In [ ]:
# Now we can concatenate the datasets, to show the difference harmonization made
neuro_harm_mri.head(3)

In [ ]:
mri_selection = MRI.drop(['Unnamed: 0'],axis=1)
top_selection = TOP.drop(['Unnamed: 0'],axis=1)
sab_selection = SABRE.drop(['Unnamed: 0'],axis=1)
    
mri_selection = mri_selection.set_index('participant_id')
top_selection = top_selection.set_index('participant_id')
sab_selection = sab_selection.set_index('participant_id')

In [ ]:
mri_selection.columns

In [ ]:
rename_dict = {
    'age': 'age_unharm', # will be the key on merge
    'sex':'sex_unharm',
    'gm_vol' :'gm_vol_unharm',
    'wm_vol' :'wm_vol_unharm',
    'csf_vol':'csf_vol_unharm',
    'gm_ivc_ratio': 'gm_ivc_ratio_unharm',
    'gmwm_ivc_ratio': 'gmwm_ivc_ratio_unharm',
    'wmh_vol' :'wmh_vol_unharm',
    'wmh_count' : 'wmh_count_unharm',
    'deepwm_b_cov':'deepwm_b_cov_unharm',
    'aca_b_cov': 'aca_b_cov_unharm',
    'mca_b_cov': 'mca_b_cov_unharm',
    'pca_b_cov': 'pca_b_cov_unharm',
    'totalgm_b_cov':'totalgm_b_cov_unharm',
    'deepwm_b':'deepwm_b_unharm',
    'aca_b':'aca_b_unharm',
    'mca_b':'mca_b_unharm',
    'pca_b':'pca_b_unharm',
    'totalgm_b':'totalgm_b_unharm', 

}
mri_selection = mri_selection.rename(columns = rename_dict)
top_selection = top_selection.rename(columns = rename_dict)
sab_selection = sab_selection.rename(columns = rename_dict)

In [ ]:
mri_selection = mri_selection.reset_index()
top_selection = top_selection.reset_index()
sab_selection = sab_selection.reset_index()
top_selection

In [ ]:
neuro_harm_mriR = neuro_harm_mri.reset_index()
neuro_harm_topR = neuro_harm_top.reset_index()
neuro_harm_sabreR = neuro_harm_sabre.reset_index()

In [ ]:
#mri_selection
neuro_harm_mriR = neuro_harm_mriR.rename(columns = {'index':'participant_id'})
neuro_harm_topR = neuro_harm_topR.rename(columns = {'index':'participant_id'})
neuro_harm_sabreR = neuro_harm_sabreR.rename(columns = {'index':'participant_id'})

In [ ]:
mri_selection.head(3)

In [ ]:
neuro_harm_mriR.head(3)

In [ ]:
The above needs correction, but is not relevant currently. Below will not run.

In [ ]:
mri_selection = mri_selection.merge(neuro_harm_mriR, how= 'inner',left_on=['participant_id'], right_on=['participant_id'])
top_selection = top_selection.merge(neuro_harm_topR,  how= 'inner',)
sab_selection = sab_selection.merge(neuro_harm_sabreR, how= 'inner',)
top_selection

In [ ]:
sab_selection

In [ ]:
mri_selection.to_csv('neuro_harm3way_mri_before_after.csv')
top_selection.to_csv('neuro_harm3way_top_before_after.csv')
sab_selection.to_csv('neuro_harm3way_sabre_before_after.csv')